**Importing the Libraries**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy import stats #for normalisation
from scipy.stats import zscore
from scipy.stats import boxcox
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
np.random.seed(40)

In [ ]:
ddata=pd.read_csv('/content/Sonar_data.csv',header=None)#since we don't have header for the coloumns
pddata=pd.DataFrame(ddata)
print(pddata.head())
print(pddata.shape)
print(pddata[60].value_counts())
print(pddata.isna().any().any())
print(pddata.dtypes)


       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   R  
1  0.0052  0.0044   R  
2  0.0095  0.0078   

In [ ]:
pddata.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [ ]:
print(pddata.iloc[:,-1].head())
pddata.groupby(60).mean()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
60,,,,,,,,,,,,,,,,,,,,,
M,0.034989,0.045544,0.050720,0.064768,0.086715,0.111864,0.128359,0.149832,0.213492,0.251022,...,0.019352,0.016014,0.011643,0.012185,0.009923,0.008914,0.007825,0.009060,0.008695,0.006930
R,0.022498,0.030303,0.035951,0.041447,0.062028,0.096224,0.114180,0.117596,0.137392,0.159325,...,0.012311,0.010453,0.009640,0.009518,0.008567,0.007430,0.007814,0.006677,0.007078,0.006024


In [ ]:
unique_values = pddata.iloc[:,-1].unique()
mapping = {value: i for i, value in enumerate(unique_values)}

# map the values in the string column to numerical values
pddata.iloc[:,-1] = pddata.iloc[:,-1].map(mapping)

# convert the column to an integer type
pddata.iloc[:,-1] = pddata.iloc[:,-1].astype(int)

# print the resulting dataframe
print(pddata.head())
print(pddata.groupby(60).mean())

       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   0  
1  0.0052  0.0044   0  
2  0.0095  0.0078   

<ipython-input-7-3a939180fbfe>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  pddata.iloc[:,-1] = pddata.iloc[:,-1].map(mapping)


In [ ]:
X=pddata.iloc[:,:-1]
Y=pddata.iloc[:,-1]





In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,stratify=Y,random_state=6)
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X,Y,test_size=0.1,stratify=Y,random_state=4)
#model=LogisticRegression(random_state=6)
lmodel = LogisticRegression(penalty='l2', solver='saga',C=9,max_iter=10000 ,random_state=6)
dtree = DecisionTreeClassifier()

model= StackingClassifier(
    estimators=[('lmodel', lmodel), ('dtree', dtree)],
    final_estimator=LogisticRegression()
)

model.fit(X_train,Y_train) # here I am using ensemble methids for increasing the accuracy of prediction


StackingClassifier(estimators=[('lmodel',
                                LogisticRegression(C=9, max_iter=10000,
                                                   random_state=6,
                                                   solver='saga')),
                               ('dtree', DecisionTreeClassifier())],
                   final_estimator=LogisticRegression())

In [ ]:
Y_tra_predict=model.predict(X_train)
acc_sco_tra=accuracy_score(Y_tra_predict,Y_train)
print(acc_sco_tra)
Y_tes_predict=model.predict(X_test)
acc_sco_tes=accuracy_score(Y_tes_predict,Y_test)
print(acc_sco_tes)
Y_tes2_predict=model.predict(X_test2)
acc_sco_tes2=accuracy_score(Y_tes2_predict,Y_test2)
print(acc_sco_tes2)

0.983957219251337
0.8571428571428571
1.0


In [ ]:
cl=classification_report(Y_train,Y_tra_predict)
print(cl)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        87
           1       0.97      1.00      0.99       100

    accuracy                           0.98       187
   macro avg       0.99      0.98      0.98       187
weighted avg       0.98      0.98      0.98       187



In [ ]:
with open("stacking_classifier","wb") as f:
  pickle.dump(model,f)

with open("stacking_classifier","rb") as f:
  mp=pickle.load(f) #here we are using pickle  for saving the model as we can use it later for different projects and we can send it to our friends
#even we can use joblib for the same fuctionality however if model has more numpy arrays we can use joblib which will be very computational

In [ ]:
arr=(0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,0.1156,0.1654,0.3833,0.3598,0.1713,0.1136,0.0349,0.3796,0.7401,0.9925,0.9802,0.8890,0.6712,0.4286,0.3374,0.7366,0.9611,0.7353,0.4856,0.1594,0.3007,0.4096,0.3170,0.3305,0.3408,0.2186,0.2463,0.2726,0.1680,0.2792,0.2558,0.1740,0.2121,0.1099,0.0985,0.1271,0.1459,0.1164,0.0777,0.0439,0.0061,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022)
parr=np.asarray(arr)
pparr=parr.reshape(1,-1)
pr=model.predict(pparr)
print(np.shape(parr))
print(np.shape(pparr))
if(pr==1):
  print("M")
else:
  print("R")

(60,)
(1, 60)
R
